In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1YRJeX9wj0o1vrpHZC7H")
project = rf.workspace("nilmi").project("tomato-plant-detection")
version = project.version(5)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Tomato-Plant-Detection-5 in yolov8:: 100%|██████████| 1234/1234 [00:00<00:00, 6753.42it/s]


In [2]:
import os
import shutil

# Your paths
source_path = '/content/Tomato-Plant-Detection-5'  # After unzipping Roboflow download
dest_path = '/content/drive/MyDrive/rover-jan-30'

# First, mount Google Drive if not already done
from google.colab import drive
drive.mount('/content/drive')

# Check if source exists
if os.path.exists(source_path):
    # Create destination folder
    os.makedirs(dest_path, exist_ok=True)

    # Move the folder
    shutil.move(source_path, dest_path)
    print("✅ Moved to /content/drive/MyDrive/rover-jan-30/Tomato-Plant-Detection-5")

    # Verify structure
    moved_path = os.path.join(dest_path, 'Tomato-Plant-Detection-5')
    print("\nDataset structure:")
    for split in ['train', 'valid', 'test']:
        img_dir = os.path.join(moved_path, split, 'images')
        lbl_dir = os.path.join(moved_path, split, 'labels')
        if os.path.exists(img_dir):
            print(f"  {split}: {len(os.listdir(img_dir))} images")
        if os.path.exists(lbl_dir):
            print(f"  {split}: {len(os.listdir(lbl_dir))} labels")

else:
    print("❌ Source not found. Check:")
    print("1. Did you unzip the Roboflow download?")
    print("2. Run: !unzip /content/*.zip -d /content/")
    print("3. Or check: !ls /content/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Moved to /content/drive/MyDrive/rover-jan-30/Tomato-Plant-Detection-5

Dataset structure:
  train: 564 images
  train: 564 labels
  valid: 24 images
  valid: 24 labels
  test: 23 images
  test: 23 labels


In [5]:
from ultralytics import YOLO
import os

dataset_path = '/content/drive/MyDrive/rover-jan-30/Tomato-Plant-Detection-5'
print("🔥 TPU v5e-1 detected but using CPU (YOLOv8 limitation)")
print("Training starts NOW...")

# Load model
model = YOLO('yolov8n.pt')

# CPU-optimized training (will work 100%)
results = model.train(
    data=f'{dataset_path}/data.yaml',
    epochs=75,                    # Balanced for CPU speed
    imgsz=416,                    # Smaller = faster CPU
    batch=8,                      # CPU-friendly batch
    name='tomato_plant_cpu_final',
    patience=15,
    save=True,
    plots=True,
    device='cpu',                 # ✅ Guaranteed to work
    workers=4                     # CPU data loading
)

print("✅ Training DONE!")
print("Model saved: /content/runs/detect/tomato_plant_cpu_final/weights/best.pt")


🔥 TPU v5e-1 detected but using CPU (YOLOv8 limitation)
Training starts NOW...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/rover-jan-30/Tomato-Plant-Detection-5/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=tomato_pl

In [6]:
import shutil
shutil.copy('/content/runs/detect/tomato_plant_cpu_final/weights/tomato.pt',
            '/content/drive/MyDrive/rover-jan-30/tomato_plant_final_best.pt')
print("✅ Model backed up to Drive!")


✅ Model backed up to Drive!
